# 🎅🤶 Advent of Code 2021 🌟☃️

https://adventofcode.com/

# Day 1

In [44]:
import pandas as pd
import os

def load_data():
    return pd.read_csv(os.path.join(".", "data", "1.csv"), header=None).squeeze()

### Part 1

In [44]:
def get_n_times_increased():
    return (load_data().diff().dropna() > 0.).aggregate(sum)

In [45]:
get_n_times_increased()

1527

### Part 2

In [54]:
def get_n_times_sum_increased():
    return (load_data().rolling(3).sum().diff() > 0.).aggregate(sum)

In [55]:
get_n_times_sum_increased()

1575

# Day 2

In [125]:
import pandas as pd
import os

def load_data():
    df = pd.read_csv(os.path.join(".", "data", "2.csv"), header=None)
    df.columns = ["raw"]
    df = df["raw"].str.split(expand = True)
    df.columns = ["direction", "magnitude"]
    df["magnitude"] = df["magnitude"].astype(int)
    return df

### Part 1

In [125]:
def get_product_of_horizontal_and_vertical_location():
    df = load_data()
    agg_df = df.groupby("direction").aggregate({"magnitude": sum})
    return (agg_df.loc["down", "magnitude"] - agg_df.loc["up", "magnitude"])*agg_df.loc["forward", "magnitude"]

In [126]:
get_product_of_horizontal_and_vertical_location()

1762050

### Part 2

In [123]:
def get_product_of_horizontal_and_vertical_location():
    df = load_data()
    for direction in df["direction"].unique():
        df[direction] = (df["direction"] == direction)*df["magnitude"] # change in each direction at each time step
    df["aim"] = df["down"].cumsum() - df["up"].cumsum()
    df["horizontal"] = df["forward"].cumsum()
    df["depth"] = (df["aim"]*df["forward"]).cumsum()
    return (df["depth"]*df["horizontal"]).values[-1]

In [124]:
get_product_of_horizontal_and_vertical_location()

1855892637

# Day 3

### Part 1

In [198]:
import os

def load_data():
    with open(os.path.join(".", "data", "3.csv")) as file:
        lines = file.readlines()
    df = pd.DataFrame([list(l) for l in lines]).drop(12, axis=1) # drop newline column
    return df
    
def get_power_consumption():
    df = load_data()
    gamma_rate = "".join(df.mode(axis=0).transpose().squeeze().values[:-1])
    epsilon_rate = "".join([{"0":"1", "1": "0"}[c] for c in gamma_rate])
    return int(gamma_rate, base=2)*int(epsilon_rate, base=2)

In [199]:
get_power_consumption()

325902

### Part 2

In [221]:
def get_life_support_rating():
    def _filter_by_bit_criterion(numbers_df, criterion, bit_i=0,):
        if criterion == "O2":
            numbers_df = numbers_df.loc[numbers_df[bit_i] == max(numbers_df[bit_i].mode()), :]
        elif criterion == "CO2":
            numbers_df = numbers_df.loc[numbers_df[bit_i] != max(numbers_df[bit_i].mode()), :]
        else:
            raise ValueError()
        
        if numbers_df.shape[0] == 1:
            return int("".join(numbers_df.iloc[0, :].transpose().squeeze().values), base=2)
        
        else:
            return _filter_by_bit_criterion(numbers_df, criterion, bit_i + 1)

    co2_scrubber_rating = _filter_by_bit_criterion(load_data(), criterion = "CO2")
    oxygen_generator_rating = _filter_by_bit_criterion(load_data(), criterion = "O2")
    return co2_scrubber_rating*oxygen_generator_rating

In [222]:
get_life_support_rating()

482500

# Day 4

In [ ]:
# Want to identify winning board, and the score of that board.
# The winning board is the board which first marks a row or column from the list of numbers.
# The score of the winning board is the sum of all of its unmarked numbers, multiplied by the
# last number that was read out. 

# Initialise a set of boards.
# Receive the first number.
# Update the board base on the number, check if any of the boards have won.
# If none of the boards have won, read the second number and repeat 3.
# If a board has won, compute the winning score and identify the winning board.

In [103]:
import numpy as np
import os
import re

class BingoBoard():
    
    def __init__(self, raw_board: list):
        # raw_board is a length-n list of str,
        # where each str contains n space-separated integers.
        self.values = np.array([line.split() for line in raw_board], dtype = int)
        self.marks = np.zeros(self.values.shape, dtype = bool)
        self.n = self.values.shape[0]
        self.in_play = True
    
    def update(self, number: int):
        if self.in_play:
            self.marks[(self.values == number)] = True
        
    def check_for_win(self, number: int):
        if self.in_play and (np.any(self.marks.sum(axis = 0) == self.n) or
                             np.any((self.marks.sum(axis = 1) == self.n))):
            self.in_play = False
            return np.sum(self.values[~self.marks])*number
        else:
            return False
        
def read_bingo_game(filepath: str):
    with open(filepath) as file:
        lines = file.readlines()
    boards = []
    current_board = []
    for j, line in enumerate(lines):
        if j == 0: # read list of called-out numbers
            called_values = np.array(line.strip().split(","), dtype = int)
        
        if j > 1: # read boards
            if (re.search("[0-9]+", line) is not None):
                current_board.append(line.strip())
            else:
                boards.append(BingoBoard(current_board))
                current_board = []
            
    return called_values, boards

def evaluate_bingo_game(called_values: list, boards: list):
    # called values is a list of integers.
    # boards is a list of BingoBoards.
    outcomes = [] # list of win indicators/board final scores
    # round of play increases going down the rows,
    # columns correspond to boards.
    for called_value in called_values:
        for board in boards:
            board.update(called_value)
        outcomes.append([board.check_for_win(called_value) for board in boards])
    return np.array(outcomes)

### Part 1

In [109]:
# First win
def get_first_win():
    called_values, boards = read_bingo_game(os.path.join(".", "data", "4.csv"))
    outcomes = evaluate_bingo_game(called_values, boards)
    n_rounds = outcomes.shape[0]
    for j in range(n_rounds):
        max_outcome = max(outcomes[j, :])
        if max_outcome > 0:
            return max_outcome
    

In [110]:
get_first_win()

49686

### Part 2

In [113]:
# Last win
def get_last_win():
    called_values, boards = read_bingo_game(os.path.join(".", "data", "4.csv"))
    outcomes = evaluate_bingo_game(called_values, boards)
    n_rounds = outcomes.shape[0]
    for j in range(outcomes.shape[0]):
        max_outcome = max(outcomes[n_rounds - 1 - j, :])
        if max_outcome > 0:
            return max_outcome

In [114]:
get_last_win()

26878

# Day 5

### Part 1

In [ ]:
# Mesh of points
# Collection of lines defined on that mesh
# Determine how many lines pass through each point on the mesh

# Decompose each line by computing the set of points it passes through,
# then, across all decompositions, compute the frequency of the points.

In [179]:
import pandas as pd
import numpy as np
import os

In [220]:
def transform_vent_coords_to_points(vent_coords: str, ignore_diagonals = True):
    EPS = 1e-10
    origin, terminus = [
        np.array(coords.split(","), dtype = int)
            for coords in re.findall(r"[0-9,]+", vent_coords)
    ]
    magnitude = np.sqrt(np.sum((terminus - origin)**2.))
    direction = np.expand_dims((terminus - origin)/magnitude, axis = -1)
    if np.all(direction > 0.7): # diagonal line
        if ignore_diagonals:
            return None
        points = np.expand_dims(origin, axis = -1) + direction*np.arange(0., magnitude + EPS, np.sqrt(2))
    else: # horizontal or vertical line
        points = np.expand_dims(origin, axis = -1) + direction*np.arange(0., magnitude + EPS, 1.)
    return points.transpose()

In [222]:
def analyse_vent_coords_collection(ignore_diagonals = True):
    with open(os.path.join(".", "data", "5.csv")) as file:
        lines = file.readlines()
        
    vent_points = [transform_vent_coords_to_points(vc, ignore_diagonals) for vc in lines]
    vent_points = np.concatenate([vp for vp in vent_points if vp is not None])
    
    # Concatenate all vent points
    return pd.Series([tuple(vent_point) for vent_point in vent_points]).value_counts()

In [225]:
transform_vent_coords_to_points(684,224 -> 83,825

array([0.        , 1.41421356, 2.82842712, 4.24264069, 5.65685425,
       7.07106781, 8.48528137, 9.89949494])

In [224]:
analyse_vent_coords_collection(ignore_diagonals = False)

(257.0, 259.0)                          4
(727.0, 744.0)                          4
(395.0, 561.0)                          4
(395.0, 589.0)                          4
(632.0, 589.0)                          4
                                       ..
(291.0, 263.0)                          1
(292.0, 264.0)                          1
(293.0, 265.0)                          1
(294.0, 266.0)                          1
(241.680628122494, 753.319371877506)    1
Length: 214775, dtype: int64

In [188]:
transform_vent_coords_to_points("364,468 -> 364,762")

array([[364, 468],
       [364, 469],
       [364, 470],
       [364, 471],
       [364, 472],
       [364, 473],
       [364, 474],
       [364, 475],
       [364, 476],
       [364, 477],
       [364, 478],
       [364, 479],
       [364, 480],
       [364, 481],
       [364, 482],
       [364, 483],
       [364, 484],
       [364, 485],
       [364, 486],
       [364, 487],
       [364, 488],
       [364, 489],
       [364, 490],
       [364, 491],
       [364, 492],
       [364, 493],
       [364, 494],
       [364, 495],
       [364, 496],
       [364, 497],
       [364, 498],
       [364, 499],
       [364, 500],
       [364, 501],
       [364, 502],
       [364, 503],
       [364, 504],
       [364, 505],
       [364, 506],
       [364, 507],
       [364, 508],
       [364, 509],
       [364, 510],
       [364, 511],
       [364, 512],
       [364, 513],
       [364, 514],
       [364, 515],
       [364, 516],
       [364, 517],
       [364, 518],
       [364, 519],
       [364,